# Clusterização com KNN

## Teste 1

Clusterização usando todas as colunas com poucos valores únicos e removendo as linhas com pelo menos um nulo.

In [ ]:
dataSelectedKNNTest1 = data[[
                     #'Data de cadastro',
                     'Tipo de violência',
                     'Violência familiar',
                     'Denunciante',
                     'Filhos vítima',
                     'Dependência Financeira da Vítima',
                     'Sexo da vítima',
                     'Raça\\Cor da vítima',
                     'Faixa etária da vítima',
                     'Grau de instrução da vítima',
                     'Frequência',
                     'Coabitação contexto',
                     'Tempo de violência contexto',
                     'Risco contexto',
                     'Relação vítima-suspeito',
                     'Sexo do suspeito',
                     'Raça\\Cor do suspeito',
                     'Faixa etária do suspeito',
                     'Grau de instrução do suspeito',
                     'Drogas Álcool Suspeito',
                     'Comportamento efeito agressor',
                     'Filhos violência',
                     #'Violações',
                     'UF',
                     #'Município',
                     'Residência'
                    ]].copy()

# Visualização dos valores nulos com um heatmap
plt.figure(figsize=(20, 10)) 
sns.heatmap(dataSelectedKNNTest1.isnull(), cbar=False, cmap='viridis')
plt.show()

numeroDeLinhasComNulo = dataSelectedKNNTest1.isnull().any(axis=1).sum()
numeroDeLinhasSemNulo = dataSelectedKNNTest1.notnull().all(axis=1).sum()
totalDeLinhas = len(dataSelectedKNNTest1)
print(f"Linhas sem valores nulos: {numeroDeLinhasSemNulo}/{totalDeLinhas} ({numeroDeLinhasSemNulo/totalDeLinhas:.2%})")

# Remove as linhas com valores nulos
dataSelectedKNNTest1 = RemoveLinhasComNulos(dataSelectedKNNTest1)

### Teste 1.1

**Codificação**: Binária

In [ ]:
# Aplica a codificação Binária
codificador = ce.BinaryEncoder(cols=dataSelectedKNNTest1.columns)
dataSelectedEncodedBinaryKNNTest1_1 = codificador.fit_transform(dataSelectedKNNTest1) 

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest1_1 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest1_1 = []
inertiaKNNTest1_1 = []

In [ ]:
# Calculando para o algoritmo 'ball_tree'
for clustersNumber in numbersOfNeighborsKNNTest1_1:
    knnBallTree = KNeighborsClassifier(n_neighbors=numbersOfNeighborsKNNTest1_1, weights='uniform', algorithm='ball_tree', p=1, metric='minkowski', metric_params=None, n_jobs=-1)
    knnBallTree.fit(dataSelectedEncodedBinaryKNNTest1_1)
    inertiaRandomTest1_1.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest1_1:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedBinaryKNNTest1_1)
    inertiaKNNTest1_1.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest1_1 = pd.DataFrame({
    "Random" : inertiaRandomTest1_1,
    "K-Means++" : inertiaKNNTest1_1,
    "Número de Clusters" : numbersOfNeighborsKNNTest1_1
})

# Visualização dos custos
figKNNTest1_1 = px.line(
    inertiasKNNTest1_1, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 1.1: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest1_1)} clusters. Dataset: {nomeDoArquivo}. Codificação: binária.<br>"
        f"Linhas com nulo removidas. Colunas Data de Cadastro, Violações e Município removidas. {info}."
    )
)

figKNNTest1_1.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização"
)

figKNNTest1_1.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest1_1.write_image(f"KNNTeste1.1{info}.png", width=1280, height=720)


In [ ]:
modelKNNTest1_1 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest1_1.fit(dataSelectedEncodedBinaryKNNTest1_1)

labelsKNNTest1_1 = modelKNNTest1_1.labels_

dataSelectedEncodedBinaryKNNTest1_1['Cluster'] = labelsKNNTest1_1

In [ ]:
#fig = px.histogram(dataSelectedEncodedBinaryKNNTest1_1, 
#                   x='Cluster',
#                   color='Faixa etária do suspeito', 
#                   barmode='group',
#                   title='Distribuição da Faixa etária do suspeito por Cluster', 
#                   labels={'Cluster': 'Cluster', 'count': 'Contagem'},
#                   color_discrete_sequence=px.colors.sequential.Viridis,
#                   #category_orders={'Faixa etária do suspeito': faixa_etaria_ordenada}
#                   )
#
#fig.update_layout(xaxis_title='Cluster', yaxis_title='Contagem', width=1000, height=600)
#fig.show()

In [ ]:
#fig = px.histogram(dataSelectedFilteredKModesTest1, x='Raça\\Cor do suspeito', color='Sexo do suspeito',barmode='group', facet_col='Cluster', 
#                   title='Distribuição da Raça\\Cor do suspeito por Cluster e Sexo', 
#                   labels={'Raça\\Cor do suspeito': 'Raça\\Cor do suspeito', 'count': 'Contagem'},
#                   color_discrete_sequence=px.colors.sequential.Viridis)
#
#fig.update_layout(xaxis_title='Raça\\Cor do suspeito', yaxis_title='Contagem', width=1000, height=600)
#fig.show()

In [ ]:
## Agrupar os dados por 'Cenário da violação' e 'Faixa etária da vítima' e calcular a soma da quantidade de vítimas
#df_summary = df_gaucho.groupby(['Cenário da violação', 'Faixa etária da vítima']).size().reset_index(name='Count')
#
## Criar uma coluna de valor fictício para o treemap
#df_summary['value'] = df_summary['Count']
#
## Criar uma coluna com os caminhos para o treemap
#df_summary['path'] = df_summary['Cenário da violação'] + " - " + df_summary['Faixa etária da vítima']
## Criar o gráfico de treemap
#fig = px.treemap(df_summary,
#                 path=['Faixa etária da vítima', 'Cenário da violação'],
#                 values='value',
#                 color='value',
#                 color_continuous_scale='Viridis',
#                 title='Distribuição por Faixa Etária da Vítima e Cenário de Violação',
#                 color_continuous_midpoint=np.average(df_summary['value']),
#                 hover_data=['path'])
#
## Ajustes no layout do gráfico
#fig.update_layout(margin=dict(t=50, l=25, r=25, b=25),height=1500)
#fig.show()

### Teste 1.2

**Codificação**: One-Hot

In [ ]:
# Aplica a codificação One-Hot
dataSelectedEncodedOneHotKNNTest1_2 = pd.get_dummies(dataSelectedKNNTest1)

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest1_2 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest1_2 = []
inertiaKNNTest1_2 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest1_2:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedOneHotKNNTest1_2)
    inertiaRandomTest1_2.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest1_2:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedOneHotKNNTest1_2)
    inertiaKNNTest1_2.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest1_2 = pd.DataFrame({
    "Random" : inertiaRandomTest1_2,
    "K-Means++" : inertiaKNNTest1_2,
    "Número de Clusters" : numbersOfNeighborsKNNTest1_2
})

# Visualização dos custos
figKNNTest1_2 = px.line(
    inertiasKNNTest1_2, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 1.2: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest1_2)} clusters. Dataset: {nomeDoArquivo}. Codificação: One-Hot.<br>"
        f"Linhas com nulo removidas. Colunas Data de Cadastro, Violações e Município removidas. {info}."
    )
)

figKNNTest1_2.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização"
)

figKNNTest1_2.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest1_2.write_image(f"KNNTeste1.2{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest1_2 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest1_2.fit(dataSelectedEncodedOneHotKNNTest1_2)

labelsKNNTest1_2 = modelKNNTest1_2.labels_

## Teste 2

Teste de clusterização sobre colunas especificas de perfil de vítima e suspeito, para as duas codificações. Colunas usadas:

 'Sexo da vítima',
 'Sexo do suspeito', 
 'Raça\Cor da vítima', 
 'Raça\Cor do suspeito',
 'Faixa etária da vítima', 
 'Faixa etária do suspeito',
 'Grau de instrução da vítima', 
 'Grau de instrução do suspeito'

In [ ]:
# Seleciona as colunas que serão utilizadas
dataSelectedKNNTest2 = data[['Sexo da vítima',
                                'Sexo do suspeito',
                                'Raça\\Cor da vítima',
                                'Raça\\Cor do suspeito',
                                'Faixa etária da vítima',
                                'Faixa etária do suspeito',
                                'Grau de instrução da vítima',
                                'Grau de instrução do suspeito']].copy()

# Visualização dos valores nulos com um heatmap
plt.figure(figsize=(20, 10)) 
sns.heatmap(dataSelectedKNNTest2.isnull(), cbar=False, cmap='viridis')
plt.show()

numeroDeLinhasComNulo = dataSelectedKNNTest2.isnull().any(axis=1).sum()
numeroDeLinhasSemNulo = dataSelectedKNNTest2.notnull().all(axis=1).sum()
totalDeLinhas = len(dataSelectedKNNTest2)
print(f"Linhas sem valores nulos: {numeroDeLinhasSemNulo}/{totalDeLinhas} ({numeroDeLinhasSemNulo/totalDeLinhas:.2%})")

# Remove as linhas com valores nulos
dataSelectedKNNTest2 = RemoveLinhasComNulos(dataSelectedKNNTest2)

### Teste 2.1

**Codificação**: Binária

In [ ]:
# Aplica a codificação Binária
codificador = ce.BinaryEncoder(cols=dataSelectedKNNTest2.columns)
dataSelectedEncodedBinaryKNNTest2_1 = codificador.fit_transform(dataSelectedKNNTest2) 

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest2_1 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest2_1 = []
inertiaKNNTest2_1 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest2_1:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedBinaryKNNTest2_1)
    inertiaRandomTest2_1.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest2_1:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedBinaryKNNTest2_1)
    inertiaKNNTest2_1.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest2_1 = pd.DataFrame({
    "Random" : inertiaRandomTest2_1,
    "K-Means++" : inertiaKNNTest2_1,
    "Número de Clusters" : numbersOfNeighborsKNNTest2_1
})

# Visualização dos custos
figKNNTest2_1 = px.line(
    inertiasKNNTest2_1, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 2.1: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest2_1)} clusters. Dataset: {nomeDoArquivo}. Codificação: binária.<br>"
        f"Linhas com nulo removidas. Colunas: Sexo, Raça\\Cor, Faixa Etária e Grau de Instrução (vítima e suspeito). {info}."
    )
)

figKNNTest2_1.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização"
)

figKNNTest2_1.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest2_1.write_image(f"KNNTeste2.1{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest2_1 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest2_1.fit(dataSelectedEncodedBinaryKNNTest2_1)

labelsKNNTest2_1 = modelKNNTest2_1.labels_

### Teste 2.2

**Codificação**: One-Hot

In [ ]:
# Aplica a codificação One-Hot
dataSelectedEncodedOneHotKNNTest2_2 = pd.get_dummies(dataSelectedKNNTest2)

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest2_2 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest2_2 = []
inertiaKNNTest2_2 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest2_2:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedOneHotKNNTest2_2)
    inertiaRandomTest2_2.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest2_2:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedOneHotKNNTest2_2)
    inertiaKNNTest2_2.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest2_2 = pd.DataFrame({
    "Random" : inertiaRandomTest2_2,
    "K-Means++" : inertiaKNNTest2_2,
    "Número de Clusters" : numbersOfNeighborsKNNTest2_2
})

# Visualização dos custos
figKNNTest2_2 = px.line(
    inertiasKNNTest2_2, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 2.2: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest2_2)} clusters. Dataset: {nomeDoArquivo}. Codificação: One-Hot.<br>"
        f"Linhas com nulo removidas. Colunas: Sexo, Raça\\Cor, Faixa Etária e Grau de Instrução (vítima e suspeito). {info}."
    )
)

figKNNTest2_2.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização"
)

figKNNTest2_2.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest2_2.write_image(f"KNNTeste2.2{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest2_2 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest2_2.fit(dataSelectedEncodedOneHotKNNTest2_2)

labelsKNNTest2_2 = modelKNNTest2_2.labels_

## Teste 3

Teste de clusterização sobre colunas especificas de perfil apenas da vítima para as duas codificações. Colunas usadas:

 'Sexo da vítima', 
 'Raça\Cor da vítima',
 'Faixa etária da vítima', 
 'Grau de instrução da vítima', 


In [ ]:
# Seleciona as colunas que serão utilizadas
dataSelectedKNNTest3 = data[['Sexo da vítima',
                                'Raça\\Cor da vítima',
                                'Faixa etária da vítima',
                                'Grau de instrução da vítima']].copy()

# Visualização dos valores nulos com um heatmap
plt.figure(figsize=(20, 10)) 
sns.heatmap(dataSelectedKNNTest3.isnull(), cbar=False, cmap='viridis')
plt.show()

numeroDeLinhasComNulo = dataSelectedKNNTest3.isnull().any(axis=1).sum()
numeroDeLinhasSemNulo = dataSelectedKNNTest3.notnull().all(axis=1).sum()
totalDeLinhas = len(dataSelectedKNNTest3)
print(f"Linhas sem valores nulos: {numeroDeLinhasSemNulo}/{totalDeLinhas} ({numeroDeLinhasSemNulo/totalDeLinhas:.2%})")

# Remove as linhas com valores nulos
dataSelectedKNNTest3 = RemoveLinhasComNulos(dataSelectedKNNTest3)

### Teste 3.1

**Codificação:** Binária

In [ ]:
# Aplica a codificação Binária
codificador = ce.BinaryEncoder(cols=dataSelectedKNNTest3.columns)
dataSelectedEncodedBinaryKNNTest3_1 = codificador.fit_transform(dataSelectedKNNTest3) 

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest3_1 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest3_1 = []
inertiaKNNTest3_1 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest3_1:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedBinaryKNNTest3_1)
    inertiaRandomTest3_1.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest3_1:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedBinaryKNNTest3_1)
    inertiaKNNTest3_1.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest3_1 = pd.DataFrame({
    "Random" : inertiaRandomTest3_1,
    "K-Means++" : inertiaKNNTest3_1,
    "Número de Clusters" : numbersOfNeighborsKNNTest3_1
})

# Visualização dos custos
figKNNTest3_1 = px.line(
    inertiasKNNTest3_1, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 3.1: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest3_1)} clusters. Dataset: {nomeDoArquivo}. Codificação: binária.  <br>"
        f"Linhas com nulo removidas. Colunas: Sexo, Raça\\Cor, Faixa Etária e Grau de Instrução (apenas vítima). {info}."
    )
)

figKNNTest3_1.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização",
)

figKNNTest3_1.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest3_1.write_image(f"KNNTeste3.1{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest3_1 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest3_1.fit(dataSelectedEncodedBinaryKNNTest3_1)

labelsKNNTest3_1 = modelKNNTest3_1.labels_

### Teste 3.2

**Codificação:** One-Hot

In [ ]:
# Aplica a codificação One-Hot
dataSelectedEncodedOneHotKNNTest3_2 = pd.get_dummies(dataSelectedKNNTest3)

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest3_2 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest3_2 = []
inertiaKNNTest3_2 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest3_2:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedOneHotKNNTest3_2)
    inertiaRandomTest3_2.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest3_2:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedOneHotKNNTest3_2)
    inertiaKNNTest3_2.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest3_2 = pd.DataFrame({
    "Random" : inertiaRandomTest3_2,
    "K-Means++" : inertiaKNNTest3_2,
    "Número de Clusters" : numbersOfNeighborsKNNTest3_2
})

# Visualização dos custos
figKNNTest3_2 = px.line(
    inertiasKNNTest3_2, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 3.2: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest3_2)} clusters. Dataset: {nomeDoArquivo}. Codificação: One-Hot.  <br>"
        f"Linhas com nulo removidas. Colunas: Sexo, Raça\\Cor, Faixa Etária e Grau de Instrução (apenas vítima). {info}."
    )
)

figKNNTest3_2.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização",
)

figKNNTest3_2.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest3_2.write_image(f"KNNTeste3.2{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest3_2 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest3_2.fit(dataSelectedEncodedOneHotKNNTest3_2)

labelsKNNTest3_2 = modelKNNTest3_2.labels_

## Teste 4

Teste de clusterização sobre colunas especificas de perfil do suspeito, para as duas codificações. Colunas usadas:

 'Sexo do suspeito',
 'Raça\Cor do suspeito', 
 'Faixa etária do suspeito', 
 'Grau de instrução do suspeito'

In [ ]:
# Seleciona as colunas que serão utilizadas
dataSelectedKNNTest4 = data[['Sexo do suspeito',
                                'Raça\\Cor do suspeito',
                                'Faixa etária do suspeito',
                                'Grau de instrução do suspeito']].copy()

# Visualização dos valores nulos com um heatmap
plt.figure(figsize=(20, 10))
sns.heatmap(dataSelectedKNNTest4.isnull(), cbar=False, cmap='viridis')
plt.show()

numeroDeLinhasComNulo = dataSelectedKNNTest4.isnull().any(axis=1).sum()
numeroDeLinhasSemNulo = dataSelectedKNNTest4.notnull().all(axis=1).sum()
totalDeLinhas = len(dataSelectedKNNTest4)
print(f"Linhas sem valores nulos: {numeroDeLinhasSemNulo}/{totalDeLinhas} ({numeroDeLinhasSemNulo/totalDeLinhas:.2%})")

# Remove as linhas com valores nulos
dataSelectedKNNTest4 = RemoveLinhasComNulos(dataSelectedKNNTest4)

### Teste 4.1

**Codificação:** Binária

In [ ]:
# Aplica a codificação Binária
codificador = ce.BinaryEncoder(cols=dataSelectedKNNTest4.columns)
dataSelectedEncodedBinaryKNNTest4_1 = codificador.fit_transform(dataSelectedKNNTest4) 

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest4_1 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest4_1 = []
inertiaKNNTest4_1 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest4_1:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedBinaryKNNTest4_1)
    inertiaRandomTest4_1.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest4_1:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedBinaryKNNTest4_1)
    inertiaKNNTest4_1.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest4_1 = pd.DataFrame({
    "Random" : inertiaRandomTest4_1,
    "K-Means++" : inertiaKNNTest4_1,
    "Número de Clusters" : numbersOfNeighborsKNNTest4_1
})

# Visualização dos custos
figKNNTest4_1 = px.line(
    inertiasKNNTest4_1, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 4.1: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest4_1)} clusters. Dataset: {nomeDoArquivo}. Codificação: binária.  <br>"
        f"Linhas com nulo removidas. Colunas: Sexo, Raça\\Cor, Faixa Etária e Grau de Instrução (apenas suspeito). {info}."
    )
)

figKNNTest4_1.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização",
)

figKNNTest4_1.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest4_1.write_image(f"KNNTeste4.1{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest4_1 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest4_1.fit(dataSelectedEncodedBinaryKNNTest4_1)

labelsKNNTest4_1 = modelKNNTest4_1.labels_

### Teste 4.2

**Codificação:** One-Hot

In [ ]:
# Aplica a codificação One-Hot
dataSelectedEncodedOneHotKNNTest4_2 = pd.get_dummies(dataSelectedKNNTest4)

# Definição de com quantos clusters será testado.
numbersOfNeighborsKNNTest4_2 = range(1, 15)

# Vetores para armazenar os "custos" (medida quantitativa de quão bem os clusters foram formados em termos de homogeneidade interna) para o K-Means
inertiaRandomTest4_2 = []
inertiaKNNTest4_2 = []

In [ ]:
# Calculando a inertia para a inicialização 'random'
for clustersNumber in numbersOfNeighborsKNNTest4_2:
    KNN_random = KNN(n_clusters=clustersNumber, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_random.fit(dataSelectedEncodedOneHotKNNTest4_2)
    inertiaRandomTest4_2.append(KNN_random.inertia_)

In [ ]:
# Calculando a inertia para a inicialização 'k-means++'
for clustersNumber in numbersOfNeighborsKNNTest4_2:
    KNN_KNN = KNN(n_clusters=clustersNumber, init='k-means++', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
    KNN_KNN.fit(dataSelectedEncodedOneHotKNNTest4_2)
    inertiaKNNTest4_2.append(KNN_KNN.inertia_)

In [ ]:
inertiasKNNTest4_2 = pd.DataFrame({
    "Random" : inertiaRandomTest4_2,
    "K-Means++" : inertiaKNNTest4_2,
    "Número de Clusters" : numbersOfNeighborsKNNTest4_2
})

# Visualização dos custos
figKNNTest4_2 = px.line(
    inertiasKNNTest4_2, 
    x = "Número de Clusters", 
    y = ["Random", "K-Means++"], 
    title = (
        f"Teste 4.2: Método do Cotovelo para o KNN com até {len(numbersOfNeighborsKNNTest4_2)} clusters. Dataset: {nomeDoArquivo}. Codificação: One-Hot.  <br>"
        f"Linhas com nulo removidas. Colunas: Sexo, Raça\\Cor, Faixa Etária e Grau de Instrução (apenas suspeito). {info}."
    )
)

figKNNTest4_2.update_layout(
    xaxis_title="Número de Clusters",
    yaxis_title="Inércia",
    legend_title="Método de Inicialização",
)

figKNNTest4_2.show()

In [ ]:
# Exportando o gráfico como uma imagem PNG
figKNNTest4_2.write_image(f"KNNTeste4.2{info}.png", width=1280, height=720)

In [ ]:
modelKNNTest4_2 = KNN(n_clusters=4, init='random', random_state=42, n_init = 1, max_iter = numeroDeEpocas, algorithm = 'lloyd', verbose=0)
modelKNNTest4_2.fit(dataSelectedEncodedOneHotKNNTest4_2)

labelsKNNTest4_2 = modelKNNTest4_2.labels_